In [30]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import matplotlib.pyplot as plt

In [2]:
aws = pd.read_csv("pr_10k.csv",sep=";",header=0)
aws.head()

,name,review,sentiment
0,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,positive
1,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,positive
2,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,positive
3,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",positive
4,A Tale of Baby's Days with Peter Rabbit,"Lovely book, it's bound tightly so you may not...",positive


In [3]:
aws = aws.drop(['name'], axis=1)
aws = aws.dropna()
polarity = aws.sentiment == "positive"

In [13]:
aws = aws.drop(['sentiment'], axis=1)
aws["polarity"] = polarity
aws.reset_index(drop = True, inplace = True)

In [18]:
print(aws.groupby(['polarity']).count())

          review
polarity        
False       2107
True        7869


In [26]:
#Définition de la performance
def show_metrics(y_true, y_score):
    # True positive
    tp = np.sum(y_true * y_score)
    # False positive
    fp = np.sum((y_true == 0) * y_score)
    # True negative
    tn = np.sum((y_true==0) * (y_score==0))
    # False negative
    fn = np.sum(y_true * (y_score==0))

    # True positive rate (sensitivity or recall)
    tpr = tp / (tp + fn)
    # False positive rate (fall-out)
    fpr = fp / (fp + tn)
    # Precision
    precision = tp / (tp + fp)
    # True negatvie tate (specificity)
    tnr = 1 - fpr
    # F1 score
    f1 = 2*tp / (2*tp + fp + fn)
    # ROC-AUC for binary classification
    auc = (tpr+tnr) / 2

    print("True positive: ", tp)
    print("False positive: ", fp)
    print("True negative: ", tn)
    print("False negative: ", fn)

    print("True positive rate (recall): ", tpr)
    print("False positive rate: ", fpr)
    print("Precision: ", precision)
    print("True negative rate: ", tnr)
    print("F1: ", f1)
    print("ROC-AUC: ", auc)

In [32]:
#Visualisation AUC-ROC
def roc_auc_score(y_true, y_pred):
    unique_vals = list(np.sort(np.unique(y_pred))) + [1]
    area_under_curve = 0
    fpr_points, tpr_points = [], []
    for i, th in enumerate(unique_vals):
        y_pred_th = np.zeros(len(y_pred))
        y_pred_th[y_pred >= th] = 1.0
        
        tp = np.sum(y_test * y_pred_th)
        fp = np.sum((y_test == 0) * y_pred_th)
        tn = np.sum((y_test==0) * (y_pred_th==0))
        fn = np.sum(y_test * (y_pred_th==0))
        tpr = tp / (tp + fn)
        fpr = fp / (fp + tn)
        fpr_points.append(fpr)
        tpr_points.append(tpr)
        
        if i > 0:
            area_under_curve += (fpr_prev - fpr) & tpr
        fpr_prev = fpr
        
    plt.figure(figsize=(5, 5), dpi=100)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")

In [20]:
#A essayer : 
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=21, stratify=y)

X = aws.review
y = aws.polarity


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=21, stratify=y)

CountVectorizer = CountVectorizer(stop_words='english',min_df = 10 )

logreg = LogisticRegression(class_weight = 'balanced', solver = 'lbfgs') 

steps = [('CountVectorizer', CountVectorizer), ('logistic_regression', logreg)]
pipeline = Pipeline(steps)

pipeline.fit(X_train, y_train) 
y_pred = pipeline.predict(X_test)



C:\Users\quent\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [28]:
show_metrics(y_test, (y_pred > 0.5).astype(np.int))

True positive:  2144
False positive:  161
True negative:  471
False negative:  217
True positive rate (recall):  0.9080897924608217
False positive rate:  0.254746835443038
Precision:  0.9301518438177874
True negative rate:  0.745253164556962
F1:  0.9189884269181312
ROC-AUC:  0.8266714785088918


C:\Users\quent\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [41]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))
    

              precision    recall  f1-score   support

       False       0.68      0.75      0.71       632
        True       0.93      0.91      0.92      2361

   micro avg       0.87      0.87      0.87      2993
   macro avg       0.81      0.83      0.82      2993
weighted avg       0.88      0.87      0.88      2993

